In [1]:
import time
import traci
import string
import random
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
import base64

In [2]:
# Define the Vehicle class
class Vehicle:
    def __init__(self, vehicle_id):
        self.vehicle_id = vehicle_id
        self.pseudonym_pool = self.generate_pseudonym_pool()
        self.current_pseudonym = self.pseudonym_pool[0]
        self.private_key, self.public_key = generate_key_pair()  # Cryptographic key pair for encryption and signing

    # Generate a pool of pseudonyms for vehicles
    def generate_pseudonym_pool(self, pool_size=10):
        return [''.join(random.choices(string.ascii_letters + string.digits, k=8)) for _ in range(pool_size)] #length is 8

    # Update the current pseudonym
    def update_pseudonym(self):
        if self.pseudonym_pool:
            self.current_pseudonym = self.pseudonym_pool.pop(0)
            print(f"[{self.vehicle_id}] Updated pseudonym to: {self.current_pseudonym}")
        else:
            print(f"[{self.vehicle_id}] Pseudonym pool exhausted. Regenerating.")
            self.pseudonym_pool = self.generate_pseudonym_pool()
            self.current_pseudonym = self.pseudonym_pool.pop(0)

In [3]:
# Vehicle Communication Methods (encryption, decryption, signing, and signature verification)
def generate_key_pair():
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    public_key = private_key.public_key()
    return private_key, public_key #returns a tuple containing public and private key


def encrypt_message(public_key, message):
    encrypted = public_key.encrypt(
        message.encode(),
        padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
    )
    return base64.b64encode(encrypted).decode() # returns base64 encrypted string


def decrypt_message(private_key, encrypted_message):
    decrypted = private_key.decrypt(
        base64.b64decode(encrypted_message),
        padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
    )
    return decrypted.decode()


def sign_message(private_key, message):
    signature = private_key.sign(
        message.encode(),
        padding.PSS(mgf=padding.MGF1(algorithm=hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
        hashes.SHA256()
    )
    return base64.b64encode(signature).decode()


def verify_signature(public_key, message, signature):
    try:
        public_key.verify(
            base64.b64decode(signature),
            message.encode(),
            padding.PSS(mgf=padding.MGF1(algorithm=hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
            hashes.SHA256()
        )
        return True
    except Exception as e:
        return False


def secure_communication(sender, receiver, message):
    encrypted_message = encrypt_message(receiver.public_key, message)
    signature = sign_message(sender.private_key, message)

    print(f"[{sender.vehicle_id}] Sending message as {sender.current_pseudonym}")
    print(f"Encrypted Message: {encrypted_message}")
    print(f"Signature: {signature}")

    decrypted_message = decrypt_message(receiver.private_key, encrypted_message)
    is_valid = verify_signature(sender.public_key, decrypted_message, signature)

    print(f"[{receiver.vehicle_id}] Received message: {decrypted_message}")
    print(f"Signature valid: {is_valid}")


In [4]:
# Periodic pseudonym update function
def periodic_pseudonym_update(vehicles, interval=5, simulation_duration=30): # updation at every 5 seconds
    start_time = time.time()
    while time.time() - start_time < simulation_duration:
        print(f"\n[Simulation Time: {int(time.time() - start_time)}s]")
        for vehicle in vehicles:
            vehicle.update_pseudonym() # updates vehicles pseudonyms for safety
        time.sleep(interval)

In [ ]:
# Start SUMO simulation and integration with VANET
def start_sumo():
    sumo_cmd = ["sumo-gui", "-c", "/Users/manidhiman/sumo/map1.sumocfg"]
    traci.start(sumo_cmd)


def simulate_vanet_with_pseudonyms():
    print("Starting SUMO...", flush=True)
    start_sumo()
    print("SUMO started successfully.", flush=True)

    vehicles = [Vehicle(f"Vehicle_{i}") for i in range(1, 4)]  # Simulate 3 vehicles
    print("Vehicles initialized.", flush=True)

    update_interval = 5  # Update pseudonyms every 5 seconds
    last_update_time = 0

    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
        print("Simulation step executed.", flush=True)

        current_time = traci.simulation.getCurrentTime() / 1000  # Convert ms to seconds
        if current_time - last_update_time >= update_interval:
            for vehicle in vehicles:
                vehicle.update_pseudonym()
            last_update_time = current_time

        # Simulate secure communication
        for sender in vehicles:
            for receiver in vehicles:
                if sender != receiver:
                    secure_communication(sender, receiver, "Traffic Data : Speed and Position")

    traci.close()
    print("Simulation completed.", flush=True)


simulate_vanet_with_pseudonyms()


Starting SUMO...
 Retrying in 1 seconds


SUMO started successfully.
Vehicles initialized.
Simulation step executed.
[Vehicle_1] Sending message as 4OK0SECQ
Encrypted Message: hDedbmVJDcf6vDoxl+AzNA22ygsILHCLkX9/0ywxVs/7KtGnRAOGC1/XXojDn53Ou8szXqV4FYQ3bT2keGyliQMvObH6xX3jqYzUFevJe3/6hIrASN7eRUlTZJmumsFo0pgz6uiOSixorHWjgOLugQP1pPD6ykDJRPSXR+4mNG+lgiObf01HPsEzQisiibtZlpdzBtNCi97jeDGgNIKnjOnnTQq7NomP/wjsdy7yV9qAwYtsnLLXjGpLhaMzFZxxOAmP813KqvHPt/GT8/qlsAh/rLhQ/V8MGHe7lDPUs6sP34HIpuYUxkY2aq/ZG0oJBAxNti3LM6z33JmpXDgdFg==
Signature: C7i2CNzeBOWlAKHCAn4I81N3FhfPpwdoZ9DKgZxuVT1jZw7b8wcO027iN5smR0DJTBmPTLh4hZOCV+X0H+Opt3LQ2JNlPUVSb26KIiyQ9BsBrEe7SDgZX5iNzicZCfQIomsee2AFCSSCFXrd/S/ESSYkQsKD+Zb3iCrt7kazri2idUCzul9aqUZKSMfOoNuGlfNIIITd1oFUAmEd6++YZk1NNF1peQduLHDgIcDn9vHYO/uD5dGn3Gwh+9WMZtUB0/uXQkMiQRHK6rg5AtATOyb1Oitu8e0IoVx8zzGN80Vk5e+Lf6L4Tv7yLk5FiK4DgWF1oIx7cdTfu119fDCnPQ==
[Vehicle_2] Received message: Traffic Data : Speed and Position
Signature valid: True
[Vehicle_1] Sending message as 4OK0SECQ
Encrypted Message: NEHgCo9DxmuGiDzIXZsX

/var/folders/gh/czf6d79n6bbgvwmp3r717qwh0000gn/T/ipykernel_62272/3441067347.py:22: UserWarning: getCurrentTime is deprecated, please use getTime which returns floating point seconds
  current_time = traci.simulation.getCurrentTime() / 1000  # Convert ms to seconds


Simulation step executed.
[Vehicle_1] Sending message as 4OK0SECQ
Encrypted Message: HmPBBIalhYKgHe9d9BkWX0tJInElxyyxel16cF4o16ASUQjh+hm3cqfl/OgSu2ZFqFgmpk6FOBPdA04Tv6HoCNMRsZFARGucCLEByJB8EszVZrtqtKov+d2StUR6iDjkZcP+lw3vbJHM3qOotG3RNIl86CP0zVAWFlzearEZH3ce7wJ6/idwckT1ookRu7g0gLTUEiVmYvZLOjZADvHmkQczFUbSkZHb3Q97q2GV3dOAjJ6wGfwhHFmTnbJ10Vo3CV2rsR5H7VFdJqFqwg7Q7bjG+HKBCXNlvCxOxGSsanfwFK6bUJNDu2KfohTiw/K0FHaA0x/vjg58VGUNRPBKsw==
Signature: kFdNWYUBOR2mdmGpXM3x6A+n83jdUDgfFLvcZSRoIozd6rNRQEs2+tzXGL3YNSNPkKsLGyYS3LCsClmTrH/jNag5ay6VJtzue/yies3ySk+GXEHo+t+h3VRvoe0aOtbHK/CtFDSchY8Hr/NWpokoOg4K6eTxlBQpZiWM+LtqbK/hrNDk2nvCXYD+rfrwAEuXlvGYJCWf48JdyV/FAv+0sgu49z84EE/85pWmf9ft34//bCinXNgtJOziJiljVrdeLzg6Pp42UsWiLV6hs9SXFIGN5+qg0UzR08d/S3bKdSUnBC8o3HM/o8VjdW8aKMJdH1vnPigiXlhyDR1y55eQTQ==
[Vehicle_2] Received message: Traffic Data : Speed and Position
Signature valid: True
[Vehicle_1] Sending message as 4OK0SECQ
Encrypted Message: qomWPrUKK4zpW5zgljneOkDmeirUrgjQM+kfUBQmdSur34CS/BaFtu8GmRsgxU/vTJMhM